## Sinary - Coding Surreal Numbers
This project demonstrates the relations between Sinary and surreal numbers. 

Code in this report requires these standard python libraries:

In [2]:
from math      import floor,log2
from random    import randint
from fractions import Fraction
import re

### Surreal numeric value list generator
The Surreal numbers are born in a particular order.
label_maker() returns the numeric value of this ordering.
In label_maker() the first returned value is the 0th surreal and it is Nan (None).

In [20]:
def label_maker (r=[Fraction(0,1)]):
    """yeilds fractions according to sureal birthday ordering.
    Nan,0,-1,1,-2,-1/2,1/2,2,-3...
    """
    yield None
    yield r[0]
    while 1:
        yield r[0] - 1
        rn = [r[0]]
        for n in r[1:]:
            m = (rn[-1]+n)/2
            yield m
            rn.extend((m,n))
        yield rn[-1]+1
        r = [rn[0]-1] + rn + [rn[-1]+1]

def labels(n):
    """returns label_maker() generator as a list to n"""
    l = label_maker()
    return [ next(l) for i in range(n) ]

label_maker() generates an ordered list...

In [4]:
birth = label_maker()
print('Order   Value')
print('=====   =====')
for i in range(16):
    print(' {:2}      {}'.format(i,next(birth)))

Order   Value
=====   =====
  0      None
  1      0
  2      -1
  3      1
  4      -2
  5      -1/2
  6      1/2
  7      2
  8      -3
  9      -3/2
 10      -3/4
 11      -1/4
 12      1/4
 13      3/4
 14      3/2
 15      3


The label_maker() code maintains a growning list of numbers in order to produce the next value. This makes is slow and computationaly expensive to find the numberic label of surreal numbers with a large birth day. It is more efficient to generate the label number directly from the order number.

### Surreal order to numberic value
This code generates a list of numeric values following surreal birth order

In [5]:
def label (x):
    """return the numeric value of this sinary as a fraction"""

    if x == 0:
        return None
    if x == 1:
        return Fraction(0)
    b = bin(x)[2:]
    s = b[1]
    wf = re.compile(r'^.({}+)(.*)$'.format(s))
    w,f = wf.findall(b)[0]
    p = 1 if s == '1' else -1
    n = p * Fraction(len(w),1)
    scale = Fraction(1,1)
    while len(f):
        scale /= 2
        if f[0] == '1':
            n +=  scale
        else:
            n += -scale
        f = f[1:]
    return n

Demonstation of surreal numeric values (labels) given random surreal order numbers...

In [6]:
for i in range(10):
    n = randint(1,10000)
    print('The {:>4}th surreal has a value of {:>10}'.format(n,str(label(n))))

The 4305th surreal has a value of   -861/256
The 5299th surreal has a value of -1689/2048
The  860th surreal has a value of    185/256
The  748th surreal has a value of    -39/256
The 6226th surreal has a value of   165/2048
The 6356th surreal has a value of   425/2048
The  354th surreal has a value of    -59/128
The  821th surreal has a value of    107/256
The 9218th surreal has a value of -4091/2048
The 4003th surreal has a value of    455/128


### Numberic value to surreal birth order

Processes for the creation of numeric values in birth order and numeric values of a given birth number are given above. But also the birth number can be determined from any given finite signed binary value.

Given an number as a dyadic, this formula calculates the matching surreal number birth order (n)

$$\large n=\frac{2^{l}\left ( s\left ( \frac{\frac{f}{2}-1}{2^{w}}+1 \right )+3 \right )-1}{2}$$

    Where:

    n = the nth surreal number having a label d
    d = a positive or negative dyadic fraction, having form: s(w+f)
    s = sign of d
    w = whole part of d
    f = fractional part of d
    l = length of register to contain n

### Equivalence Equation
This formula shows the equivelence between surreal birth order and its numeric value (mod 2):

$$\large \frac{2×order+1}{2^{bits}}-2 = sign \left( \frac{fraction - 2}{2^{whole + 1}}+1 \right) +1$$

$$\large \frac{2n+1}{2^{b}}-2 = s \left( \frac{f-2}{2^{w+1}}+1 \right) +1$$

    where:
    
    given: any surreal birth ordered number having a dyadic numberic value...

    n = order    = surreal birth order
    b = bits     = binary register size of order
    s = sign     = sign of the value
    f = fraction = fractional portion of the value
    w = whole    = whole portion of the value

### Equivelance code
This routine verifies the equivelence...

In [7]:
def left_equality(n,b):
    return (2*n+1)/2**b-2

def right_equality(s,w,f):
    return s*((f/2-1)/2**w+1)+1

def equality(t,s,x,m):
    return s*((2**t*m+t)/(2**x)+1)-2*t-1

def generic_left_equality(n,b):
    return equality(+1,+1,b,n)

def generic_right_equality(s,w,f):
    return equality(-1,s,w,f)

The left_equality() and right_equality() routines match the above formula.

The equality() and generic_X() routines demonstrate the symmetric nature of the equality formula. The symmetry shows that the two sides of the equation are evaluate in the same manner with only input values as diffrentiators. The side parameter is defined as the unit negation of the other.

This code evaulates the two sides of the equality for given surreal number births and values...

In [8]:
in_sequence = label_maker()
print('\n                                     Left Equality   Right Equality')
for n in range(2**4):
    d = next(in_sequence)
    if d is None: continue
    b = n and floor(log2(n))
    a = abs(d)
    s = d and a/d
    w = floor(a)
    f = a - w
    print(' Given the{:3}th sureal is {:>4}    =>   {:>6}               {}'.format(n,str(d),left_equality(n,b),right_equality(s,w,f)))
    assert(left_equality(n,b)==right_equality(s,w,f))
    assert(generic_left_equality(n,b)==generic_right_equality(s,w,f))
    assert(equality(+1,+1,b,n)==equality(-1,s,w,f))
print("\nsuccess: all of the above is True")


                                     Left Equality   Right Equality
 Given the  1th sureal is    0    =>      1.0               1
 Given the  2th sureal is   -1    =>      0.5               1/2
 Given the  3th sureal is    1    =>      1.5               3/2
 Given the  4th sureal is   -2    =>     0.25               1/4
 Given the  5th sureal is -1/2    =>     0.75               3/4
 Given the  6th sureal is  1/2    =>     1.25               5/4
 Given the  7th sureal is    2    =>     1.75               7/4
 Given the  8th sureal is   -3    =>    0.125               1/8
 Given the  9th sureal is -3/2    =>    0.375               3/8
 Given the 10th sureal is -3/4    =>    0.625               5/8
 Given the 11th sureal is -1/4    =>    0.875               7/8
 Given the 12th sureal is  1/4    =>    1.125               9/8
 Given the 13th sureal is  3/4    =>    1.375               11/8
 Given the 14th sureal is  3/2    =>    1.625               13/8
 Given the 15th sureal is    3    =

### Equivelence Graph
The graph below displays the left side of the equivalence equation in black and the right side of the equaltion in green.
The black line has Integer input and it is only valid at whole number values.
The green line has real continuous input.

<img src="images/graph.jpg" width="800">
The graph shows the results for the first few surreal inputs. The dark dots are birth order numbers. Green dots are dyadic rational surreal values.
<img src="images/graph2.jpg" width="800">
The birth order and numeric value of surreal numbers are not in the same order, so they can not be displayed over top of each other directly. However, the graph shows their equality in height. They are equal when their different inputs result in equal outputs. Remember that the green line is continuous (dyadics), but the dark line is only valid at the integer points. When the green line drops, the value appears at the bottom of the dip.


The height of the dark dots will be equal to some green dot on the green line.
The green dot on the green line is the dyatic value. That dyatic will be the value of the surreal number which is read off the y-axis.
<img src="images/graph4.jpg" width="800">

### Surreal order and numeric value points on a circle
The equality values are unique in a range of 2. These values can be plotted onto the circle with its imaginary pi product raised to the natural exponent.

This allows the rotational constants to be removed and makes for a real sweet equation...

$$\large e^{\pi i \times \frac{2×order+1}{2^{bits}}} = e^{\pi i \times sign \frac{ fraction/2 - 1}{2^{whole}}}$$

$$\large e^{\pi i \frac{2n+1}{2^{b}}} = e^{\pi i s \frac{f/2-1}{2^{w}}}$$

<img src="images/birth1-sm.jpg" width="600">
<img src="images/birth2-sm.jpg" width="600">
<img src="images/birth3-sm.jpg" width="600">


## Sinary Numbers
Sinary numbers ARE the birth order number of Surreal Numbers.

The last sentence is very important and you should read it again. Because it says:

SINARY = SURREAL BIRTH NUMBER

The capitalization means it is an important fact that took some consideration to appreciate. This fact was not mentioned in any text or online where I could find it. The essence of this statement says that all surreal numbers have a particular numeric order in which they were born which has a binary representation that can be manipulated directly according to the mathematical operation rules provided by John Conway.

Since I could not find a specific mention of handling binary surreal birth order numbers as a unique bit format, I named them "Sinary".

With that powerful idea, there are several simple routines for manipulating sinary numbers in a surreal manner.

### Sinary value
A sinary number is the binary representation of its surreal birth order. Since the binary number evalues to the birth order on a computer (numbers are binary representations in a computer), the value of a sinary is determined using the label() routine.

In [9]:
for i in range(7):
    sinary = randint(1,10000)
    print('The Sinary {:>4} has a value of {:>10}'.format(sinary,str(label(sinary))))

The Sinary 6334 has a value of   381/2048
The Sinary 9400 has a value of -3727/2048
The Sinary 6782 has a value of  1277/2048
The Sinary 3191 has a value of   239/1024
The Sinary 6415 has a value of   543/2048
The Sinary 5015 has a value of -1233/1024
The Sinary 6060 has a value of  -167/2048


### Left and Right sides of a Sinary
The left and right side of a sinary are easily determined as reduced copies of the original sinary.

In [35]:
def side(s,f):

    "returns the lessor side of s"
    if s in (0,1) and f is 1:
        return 0
    while s % 2 != f:
        s //= 2
    return s//2

def split(s):
    "list of the lessor and greater split of s"

    return side(s,1), side(s,0)

side() is a generic function which will return the left (f=1) or right (f=0) side of the given sinary.

split() returns both the left and right side of a given sinary value.

The code below displays the left and right sinaries of the first few sinaries...

In [36]:
for sinary in range(10):
    print(sinary,split(sinary))

0 (0, 0)
1 (0, 0)
2 (0, 1)
3 (1, 0)
4 (0, 2)
5 (2, 1)
6 (1, 3)
7 (3, 0)
8 (0, 4)
9 (4, 2)


The above displays the left/right paired sides for each sinary, which matches the surreal pair values.

The equivelence to surreal numbers can be shown by evaluating the right and left sinary pairs as numeric values...

In [12]:
for sinary in range(10):
    print('\n{}th sinary is {} which has (R,L) of {}'.format(sinary,label(sinary),tuple(str(label(x)) for x in split(sinary))))


0th sinary is None which has (R,L) of ('None', 'None')

1th sinary is 0 which has (R,L) of ('None', 'None')

2th sinary is -1 which has (R,L) of ('None', '0')

3th sinary is 1 which has (R,L) of ('0', 'None')

4th sinary is -2 which has (R,L) of ('None', '-1')

5th sinary is -1/2 which has (R,L) of ('-1', '0')

6th sinary is 1/2 which has (R,L) of ('0', '1')

7th sinary is 2 which has (R,L) of ('1', 'None')

8th sinary is -3 which has (R,L) of ('None', '-2')

9th sinary is -3/2 which has (R,L) of ('-2', '-1')


### Sinary value ordering and comparison
Sinary numbers the not numerically ordered. However, Sinary numbers can be compared using Conways rules of comparison and operation on surreal numbers.

Given numeric forms x = { XL | XR } and y = { YL | YR }, x ≤ y if and only if:

- there is no xL ∈ XL such that y ≤ xL (every element in the left part of x is smaller than y), and
- there is no yR ∈ YR such that yR ≤ x (every element in the right part of y is bigger than x).

This code compares two sinary numbers...

In [13]:
def le(x,y):
    return sle(split(x),split(y))

def sle(x,y):
    "less than or equal to comparsion of x and y split representations"

    return not(x[0] and sle(y,split(x[0])) or y[1] and sle(split(y[1]),x))

The le() routine returns the truth of a "less than or equal to" comparison of two sinary numbers.

The sle() routine is recursive and expects a split list representation of the two numbers.

This code compares all truth combinations of the first 32 sinaries against the truth comparison on the numeric values of these sinaries...

In [42]:
c = 2**7
l = labels(c*c)
i = 0
for a in range(1,c):
    for b in range(1,c):
        assert(le(a,b) == (l[a]<=l[b]))
        i += 1
print("Success: {} sinary numbers compared directly".format(i))
    

Success: 16129 sinary numbers compared directly


### Additional Comparisons
All other comparison between sinary numbers is arranged logically as a "less than or equal" comparison...

In [45]:
def le(x,y):
    return sle(split(x),split(y))

def gt (x,y):
    """True if x is greater than y"""
    return not le(x,y)

def ge (x,y):
    """True if x is greater than or equal to y"""
    return le(y,x)

def lt (x,y):
    """True if x is less than y"""
    return not le(y,x)

def ne (x,y):
    """True if x is not equal to y"""
    return not (le(x,y) and le(y,x))

def eq(x,y):
    "true if x and y are equal"
    return le(x,y) and le(y,x)

def seq(x,y):
    "true if x and y are equal, where x and y are split representations"
    return sle(x,y) and sle(y,x)

### Sinary Parent
Sinary number consist of a left and right sinary number pairs. The pairs are determined from a given sinary above. This demonstrates the process of determining the partent of its given sinary pairs...

In [56]:
def join (xl,xr,y=1):
    """given a left and right sinary, return its equivelent single form"""

    X = (xl,xr)
    Y = split(y)
    while not seq(X,Y):
        y *= 2
        if sle(Y,X): y += 1
        Y = split(y)
    return y

The join() routine works, but requires comparisons to be performed.

The author believes that there is a short cut for joining two sinaries when they are in reduced form. Reduced form is the form in which the left and right sinary values of your number are the same as produced from the routines above. However, non-reduced forms occur during operational manipulations. In these cases, the join() command will need to find the parent using this inefficient searching method.

I think: Either the left or right sinary will be the parent sinary with the last bit trimmed off. And the bit trimmed will be opposite to the bit which remains. So the parent can be found by taking the longest of the two pairs and attaching a bit that is not the existing trailing bit of this longest of pairs. Easy right? But I have not tested yet and this join is used often in situations where the sinary is not in reduced form. Anyhow... thought I would mention this in case you want to research it.

Thoughts: join() is required during addition and other operations. The efficient version could be tried first. If it doesn't work or produces results that do not match the splitting of the result, then the long form with search would be used. I think this would reduce recussion and improve speed.

This code test the join() routine on the first 1024 sinary numbers...

In [81]:
print("\nSinary   Split   Joined\n")
for sinary in range(10):
    if sinary in (None,0): continue
    print('  {}     {}      {}'.format(sinary, split(sinary),join(*split(sinary))))
    assert(sinary==int(join(*split(sinary))))
print("\nSuccess: joining the split values returns the original sinary")


Sinary   Split   Joined

  1     (0, 0)      1
  2     (0, 1)      2
  3     (1, 0)      3
  4     (0, 2)      4
  5     (2, 1)      5
  6     (1, 3)      6
  7     (3, 0)      7
  8     (0, 4)      8
  9     (4, 2)      9

Success: joining the split values returns the original sinary


### Sinary Addition
Sinary addition is performed in the surreal manner...

Surreal number addition is defined by...
$$x+y=\{X_{L}|X_{R}\}+\{Y_{L}|Y_{R}\}=\{X_{L}+y,x+Y_{L}|X_{R}+y,x+Y_{R}\}$$

where...
$$X+y=\{x+y:x\in X\},x+Y=\{x+y:y\in Y\}$$

with the condition that...

- an input is the output if the other input is zero (label of one)
    
If one input has the value zero return the other because numerically: a+0 = a

This code demonstrates the addition process

In [ ]:
def add (x,y):
    "add two sinary numbers"
    if x is 1:
        return y
    if y is 1:
        return x
    xl,xr = split(x)
    yl,yr = split(y)
    left  = xl and add(xl,y)
    right = xr and add(xr,y)
    if yl:
        less = add(x,yl)
        if not left or le(left, less):
            left = less
    if yr:
        more = add(x,yr)
        if not right or le(more, right):
            right = more
    return join(left,right)

the add() routine checks for special conditions (x or y is 1) before recursing through the addition steps defined by surreal number addition. It uses the join command to return the sinary parent of the left and right split sides of the addition process.

The add() routine takes two sinary numbers and splits them into pairs to get 4 numbers. It then performs comparisons and addition on the inputs and with split numbers. When calling addition in this process, the entire process is repeated. Each split number is a reduction of its parent number and will specifically have less bits in its representation. In this way all inputs will eventually reduce to the sinary representation of "1" where the recursion will stop and the return values will be sent back to continue on with processing.

interesting note: sinary numbers split in to two numbers which are always smaller to write in binary and they will all reduce to "one" after some number of reduction steps... a binary bit digit and nothing else.

### Sinary Negation
Negation of a given number x = { XL | XR } is defined by

$${\displaystyle -x=-\{X_{L}|X_{R}\}=\{-X_{R}|-X_{L}\}}-x=-\{X_{L}|X_{R}\}=\{-X_{R}|-X_{L}\}$$

where the negation of a set S of numbers is given by the set of the negated elements of S:

$${\displaystyle -S=\{-s:s\in S\}}-S=\{-s:s\in S\}$$

But the negation of a Sinary can be interpreted as a binary involution, because it flips values in a manner that returns the value to the original state when performed twice. 

In [82]:
def mask(x,m=0):
    "generate a binary number of all '1's up to one less than the bits of x"
    while x>>1:
        x = x>>1
        m = (m<<1)|1
    return m

def neg(x):
    "flip all bits except the first"
    return x and x^mask(x)

### Sinary Subtraction
Subtraction is the addition of the negation.

$$a-b = a+(-b)$$

This code performs sinary subtraction...

In [83]:
def sub (x,y):
    return add(x,neg(y))

### Sinary multiplication

In [84]:
def mul (x,y):
    "multiply two sinary numbers"
    if 0 in (x,y): return 0
    if x is 1 or y is 3 : return x
    if y is 1 or x is 3 : return y
    if x is 2 : return -y
    if y is 2 : return -x
    xl,xr = split(x)
    yl,yr = split(y)
    left,right = 0,0
    if xl and yl:
        left  = sub(add(mul(xl,y),mul(x,yl)),mul(xl,yl))
    if xr and yr:
        left2 = sub(add(mul(xr,y),mul(x,yr)),mul(xr,yr))
        if left == 0 or not le(left2,left):
            left = left2
    if xl and yr:
        right  = sub(add(mul(xl,y),mul(x,yr)),mul(xl,yr))
    if xr and yl:
        right2 = sub(add(mul(x,yl),mul(xr,y)),mul(xr,yl))
        if right == 0 or not le(right,right2):
            right = right2
    return reduce(left,right)

### Sinary Inversion
Inversion is a long recursive operation defined by:

For positive ${\textstyle y}$,

$${\displaystyle {\frac {1}{y}}={\Bigg \{}0,{\frac {1+(y_{R}-y)({\frac {1}{y}})_{L}}{y_{R}}},{\frac {1+(y_{L}-y)({\frac {1}{y}})_{R}}{y_{L}}}{\Bigg |}{\frac {1+(y_{L}-y)({\frac {1}{y}})_{L}}{y_{L}}},{\frac {1+(y_{R}-y)({\frac {1}{y}})_{R}}{y_{R}}}{\Bigg \}}}$$

where only positive ${\textstyle y_{L}}{\textstyle y_{L}}$ are permitted in the formula, with any nonpositive terms being ignored (and ${\textstyle y_{R}}$ ${\textstyle y_{R}}$ are always positive).

For negative ${\textstyle y}$,

$${\displaystyle {\frac {1}{y}}=-\left({\frac {1}{-y}}\right)}$$


Along with the following special conditions...

If ${\textstyle y = 1}$ then ${\displaystyle {\frac {1}{y}}= 1}$

If ${\textstyle y = 0}$ then ${\displaystyle {\frac {1}{y}}= \text{undefined}}$

In [94]:
def invert (y):
    """return 1/y, the inversion from input y"""
    if   eq(y,pos) : return y
    elif lt(y,nil) : return neg(invert(neg(y)))
    elif eq(y,nil) : raise  ZeroDivisionError()
    yl,yr = split(y)
    il = nil
    ir = None
    r  = None,None
    iyr,iyl = None,None
    while (il or ir):
        nl = nr = None
        if il is not None:
            r = (il,r[1])
            if yr is not None:
                if iyr is None:
                    iyr = invert(yr)
                left = mul(mul(add(pos,sub(yr,y)),il),iyr)
                if r[0] is None or gt(left,r[0]):
                    nl = left
            if yl is not None and not le(yl,nil):
                if iyl is None:
                    iyl = invert(yl)
                right = mul(mul(add(pos,sub(yl,y)),il),iyl)
                if r[1] is None or lt(right,r[1]):
                    nr = right
        if ir:
            r = (r[0],ir)
            if yl is not None and not le(yl,nil):
                if iyl is None:
                    iyl = invert(yl)
                left = mul(mul(add(pos,sub(yl,y)),ir),iyl)
                if r[0] is None or (gt(left,r[0]) and (not nl or gt(left,nl))):
                    nl = left
            if yr is not None:
                if iyr is None:
                    iyr = invert(yr)
                right = mul(mul(add(pos,sub(yr,y)),ir),iyr)
                if r[1] is None or (lt(right,r[1]) and (not nr or lt(right,nr))):
                    nr = right
        il,ir = nl,nr
    if r[0] is None: r = (0,r[1])
    if r[1] is None: r = (r[0],0)
    return join(*r)

### Sinary Division

Since $\frac{x}{y} = x \times \frac{1}{y}$, likewise division is defined as the product of applying the inverse over its second input...

In [95]:
def divide(x,y):
    return mul(x,invert(y))

### Conclusions
Sinary numbers have a bit format that is equivalent to the surreal birth order number.

Sinary numbers have short binary manipulations which produce equivelent results to the documented surreal comparisons, operations and manipulations.

Sinary code provides a method for mathematical operations and value calculations on a bit format that is NOT standard binary.